### Tutorial 3: Adding a new diatomic species entry to your local data store

If you would like to do calculations using a plasma species that you haven't used before, you will need to generate a minplascalc data entry for it first. You only need to do this <i>once</i> - minplascalc will store the species data in a plain-text file formatted using JSON syntax, and it will then be available for use in any of your future calculations.

The procedure for generating a minplascalc data entry for a new diatomic species is entirely manual since the data required must be retrieved from various locations. The majority of data for a wide range of species is available from the NIST Chemistry WebBook, http://webbook.nist.gov/chemistry/form-ser/. Enter the formula for the diatomic species you're interested in in the search box at the top of the page, and click the Search button. If you're looking for data for charged diatomic ions, make sure the "Exclude ions from the search" box is unchecked. Let's build a data entry for Silicon Monoxide, SiO:

<table>
<td><img src="demoimg/webbook_landingpage.png" alt="NIST Chemistry WebBook" style="height: 300px;"/>
<td><img src="demoimg/webbook_sio.png" alt="SiO WebBook entry" style="height: 300px;"/>
</table>

The first pieces of information needed are the ionisation energy (to turn SiO into SiO<sup>+</sup>) and the bond dissociation energy (to turn SiO into monatomic Si and O), both in units of cm<sup>-1</sup>. The ionisation energy can be found on the "Gas phase ion energetics" link on the Chemistry WebBook species page (usually in units of eV, so remember to convert):

<img src="demoimg/webbook_ionenergetics.png" alt="SiO ionisation energy" style="width: 400px;"/>

The dissociation energy can be found either on the NIST Computational Chemistry Comparison and Benchmark Database (http://cccbdb.nist.gov/introx.asp), or alternatively from chemistry textbooks or other sources. In the case of SiO, the value is 66707.6 cm<sup>-1</sup> (approximately 800 kJ/mol).

We then need some information about the electronic ground state of the molecule, and its vibrational and rotational parameters. To get this, go back to the main page for SiO in the Chemistry WebBook and click on "Constants of diatomic molecules". This gives the parameters of various energetic states of the molecule - we want the ground state with energy $T_e$ = 0, so scroll all the way down to the entry at the very bottom:

<table>
<td><img src="demoimg/webbook_constants1.png" alt="Molecule constants" style="height: 250px;"/>
<td><img src="demoimg/webbook_constants2.png" alt="Ground state level entry" style="height: 250px;"/>
</table>

The electronic degeneracy of the ground state, $g_0$, is generally given by the superscripted number in the orbital expression in the "State" column - in the case of SiO, it's 1. The vibrational constant $\omega_e$ and rotational constant $B_e$ are read off the appropriate columns for the ground state entry. The symmetry constant $\sigma_s$ takes the value of 1 for heteronuclear molecules like SiO, and 2 for homonuclear molecules like O<sub>2</sub>. Now we have all the information to build a minplascalc data entry for the SiO species, which can be done by running the following code snippet:

In [1]:
import minplascalc as mpc
from scipy import constants

invcm_to_joule = constants.Boltzmann / (0.01*constants.value('Boltzmann constant in inverse meters per kelvin'))

siliconoxide = mpc.DiatomicSpecies(
    name='SiO',
    stoichiometry={'Si': 1, 'O': 1},
    molarmass=0.0440849,
    chargenumber=0,
    ionisationenergy=mpc.constants.invcm_to_joule*92673.11,
    dissociationenergy=mpc.constants.invcm_to_joule*66707.6,
    sigma_s=1,
    g0=1,
    w_e=mpc.constants.invcm_to_joule*1241.55,
    b_e=mpc.constants.invcm_to_joule*0.7267512,
    sources=['NIST Chemistry WebBook, NIST Standard Reference Database Number 69. '
             'PJ Linstrom and WG Mallard (Editors), National Institute of Standards '
             'and Technology, Gaithersburg MD., http://webbook.nist.gov/chemistry/, '
             'doi:10.18434/T4D303'])

mpc.species_to_file(siliconoxide)

What's happening here? First we import the minplascalc package, then we create a `DiatomicSpecies` object using our data, and finally we save the contents to a file.

The `DiatomicSpecies` class constructor takes as arguments the species name, a small dictionary describing the elemental stoichiometry of the species (in this case one oxygen and one silicon), the molar mass in kg/mol, the charge on the species in units of the fundamental charge (in this case 0 because SiO is a neutral species), the ionisation and dissociation energies of the species in J, and the electronic, vibrational, and rotational energy level parameters (with all energies in J), and a list of data sources for referencing. The object can be written out to disk using the `species_to_file` utility function, which saves data to a JSON-formatted file. It takes two arguments: a minplascalc Species object, and an optional filename - if omitted, the filename is created using the species' name.

After this process it will be possible to create an SiO species object in any minplascalc calculation by importing it using either the explicit path to the JSON file, or (preferably) just the name of the species provided the JSON file is stored in any of the standard minplascalc data paths - see later demos for examples.